In [ ]:
# Monto il drive Google

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install codecarbon

# Modello MobileNet-V3 Small + ConvLSTM (5° Split)


*   Peso modello: circa 12 MB
*   Accuratezza modello: 95 %
*   Auc: circa 99 %
*   Tempo inferenza medio per batch: 0.445 s
*   Consumo Energetico medio per 102 secondi di video:  0.001762  kW/h

In [ ]:
# Informazioni sul peso del modello

import os

MODEL_PATH_CONV = "/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Quantizzazione Aware Training/ConvLSTm/final_model_fold_5.tflite"


if os.path.exists(MODEL_PATH_CONV):
    file_size_bytes = os.path.getsize(MODEL_PATH_CONV)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"Dimensione del file: {file_size_mb:.2f} MB")
else:
    print("Il file specificato non esiste.")

In [ ]:
# Funzioni di preprocessamento dei video e di inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH_CONV)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels



from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_QAT")
def inferenceV3_ConvLSTM():
    MODEL_PATH_CONV = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Quantizzazione Aware Training/ConvLSTm/final_model_fold_5.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(MODEL_PATH_CONV)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm_QAt')

if __name__ == '__main__':
    inferenceV3_ConvLSTM()


# Modello MobileNet-V3 Small + BiLstm (5° Split)


*   Peso modello: circa 114.5 MB
*   Accuratezza modello: 81.77 %
*   Auc: circa 99.82 %
*   Tempo inferenza medio per batch: 0.38 s
*   Consumo Energetico medio per 102 secondi di video: 0.001538  kW/h

In [ ]:
# Carico il modello, ho salvato i pesi .h5 dell'aware training quindi occorre fare un
# passaggio extra per riconvertirlo in tflite

#import tensorflow_model_optimization as tfmot
import tensorflow as tf
from tensorflow.keras import layers
import keras


def load_model_as_tflite(model):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    converter.target_spec.supported_ops = [
      tf.lite.OpsSet.TFLITE_BUILTINS,
      tf.lite.OpsSet.SELECT_TF_OPS
    ]
    # Disabilita le operazioni non supportate
    converter._experimental_lower_tensor_list_ops = False

    tflite_model = converter.convert()

    return tflite_model


In [ ]:
# Informazioni sul peso del modello

MODEL_PATH = "/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Quantizzazione Aware Training/BiLSTM/BiLSTM_tfLite_ultimo_split.tflite"


if os.path.exists(MODEL_PATH):
    file_size_bytes = os.path.getsize(MODEL_PATH)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"Dimensione del file: {file_size_mb:.2f} MB")
else:
    print("Il file specificato non esiste.")

In [ ]:
# Funzioni di preprocessamento e di inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_BiLSTM_QAT")
def inferenceV3_BiLSTM():
    MODEL_PATH = '/content/model_quantized.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(MODEL_PATH)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm_QAt')

if __name__ == '__main__':
    inferenceV3_BiLSTM()
